# 04 Optimize

In [4]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
from optimize import optimize_tourney

In [5]:
df = pd.read_csv('../data/opt_input.csv')
df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,Strong_Win,Orig_Seed_Strong,Orig_Seed_Weak,Matchup,Round_v2,comp,Key,Win_Prob,Win_Prob_glm
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,1,1,16,15,1,1,115,0.992647,0.924092
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,0,1,16,15,1,-1,115,0.007353,0.924092
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,1,1,8,7,2,1,207,0.811594,0.762481
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,0,1,8,7,2,-1,207,0.188406,0.762481
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,1,16,8,8,2,-1,208,0.254310,0.208672


In [6]:
# weight win prob by round
df['Win_Prob_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob'], df['Round'])]
df['Win_Prob_glm_weighted'] = [w*(2**(r-1)) for w,r in zip(df['Win_Prob_glm'], df['Round'])]

df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,Orig_Seed_Strong,Orig_Seed_Weak,Matchup,Round_v2,comp,Key,Win_Prob,Win_Prob_glm,Win_Prob_weighted,Win_Prob_glm_weighted
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,1,16,15,1,1,115,0.992647,0.924092,0.992647,0.924092
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,...,1,16,15,1,-1,115,0.007353,0.924092,0.007353,0.924092
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,1,8,7,2,1,207,0.811594,0.762481,1.623188,1.524963
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,...,1,8,7,2,-1,207,0.188406,0.762481,0.376812,1.524963
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,...,16,8,8,2,-1,208,0.254310,0.208672,0.508621,0.417343


In [7]:
opt_results = optimize_tourney(df, 'Win_Prob_glm_weighted', 'max')

In [8]:
opt_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,1395_6,1138_6,1397_6,1400_6,1401_6,1274_6,1403_6,1276_6,1277_6,Chosen
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,0,0,0,0,0,0,0,0,0,1.0
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,0,0,0,0,0,0,0,0,0,1.0
8,2018,R1W8,W08,W09,1,1439,Virginia Tech,1104,Alabama,1439,...,0,0,0,0,0,0,0,0,0,1.0
22,2018,R1W4,W04,W13,1,1455,Wichita St,1267,Marshall,1455,...,0,0,0,0,0,0,0,0,0,1.0
42,2018,R3W1,R2W1,R2W4,3,1437,Villanova,1293,Murray St,1437,...,0,0,0,0,0,0,0,0,0,1.0


In [9]:
opt_results.to_csv('../data/opt_results.csv', index=False)

In [10]:
df.to_csv('../data/opt_input_v2.csv', index=False)